# Welcome to Exkaldi

In this section, we will build a decision tree. In order to train a triphone model, a decision tree is necessary.

In [1]:
import exkaldi

import os
dataDir = "librispeech_dummy"

Restorage lexicons generated in early step (3_prepare_lexicons).

In [2]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.load_lex(lexFile)

Then instantiate a __DecisionTree__ object. ___lexicons___ can be provided as a parameter.

In [3]:
tree = exkaldi.hmm.DecisionTree(lexicons=lexicons, contextWidth=3, centralPosition=1)

tree

Then prepare acoustic feature, hmm model and alignment.

In [4]:
featFile = os.path.join(dataDir, "exp", "train_mfcc_cmvn.ark")
feat = exkaldi.load_feat(featFile)
feat = feat.add_delta(order=2)

feat.dim

39

Monophone HMM model and alignment have been generated in last step (5_train_mono_HMM-GMM). Now use them directly. In terms of all archives, that are feature, CMVN, probability, fmllr and alignment, we do not allow you use their file directly. So you need load them.

You can load the data or only load the index table.

In [5]:
hmmFile = os.path.join(dataDir, "exp", "train_mono", "final.mdl")

aliFile = os.path.join(dataDir, "exp", "train_mono", "final.ali")
ali = exkaldi.load_index_table(aliFile, useSuffix="ark")
ali

{'103-1240-0000': IndexInfo(frames=1407, startIndex=0, dataSize=7056, filePath='librispeech_dummy/exp/train_mono/final.ali'),
 '103-1240-0001': IndexInfo(frames=1593, startIndex=7056, dataSize=7986, filePath='librispeech_dummy/exp/train_mono/final.ali'),
 '103-1240-0002': IndexInfo(frames=1393, startIndex=15042, dataSize=6986, filePath='librispeech_dummy/exp/train_mono/final.ali'),
 '103-1240-0003': IndexInfo(frames=1469, startIndex=22028, dataSize=7366, filePath='librispeech_dummy/exp/train_mono/final.ali'),
 '103-1240-0004': IndexInfo(frames=1250, startIndex=29394, dataSize=6271, filePath='librispeech_dummy/exp/train_mono/final.ali'),
 '103-1240-0005': IndexInfo(frames=1516, startIndex=35665, dataSize=7601, filePath='librispeech_dummy/exp/train_mono/final.ali'),
 '103-1240-0006': IndexInfo(frames=956, startIndex=43266, dataSize=4801, filePath='librispeech_dummy/exp/train_mono/final.ali'),
 '103-1240-0007': IndexInfo(frames=1502, startIndex=48067, dataSize=7531, filePath='librispeech_

As training the HMM model, we provide high-level API to train tree, but now we still introduce the training steps in detail.

### Train Dicision Tree in detail

#### 1. Accumulate statistics data

In [6]:
outDir = os.path.join(dataDir, "exp", "train_delta")

exkaldi.utils.make_dependent_dirs(outDir, False)

In [7]:
treeStatsFile = os.path.join(outDir, "treeStats.acc")

tree.accumulate_stats(feat, hmmFile, ali, outFile=treeStatsFile)

'librispeech_dummy/exp/train_delta/treeStats.acc'

#### 2. Cluster phones and compile questions.

In [8]:
topoFile = os.path.join(dataDir, "exp", "topo")

questionsFile = os.path.join(outDir, "questions.qst")

tree.compile_questions(treeStatsFile, topoFile, outFile=questionsFile)

'librispeech_dummy/exp/train_delta/questions.qst'

#### 3. Build tree.

In [9]:
targetLeaves = 300

tree.build(treeStatsFile, questionsFile, topoFile, numLeaves=targetLeaves)

Decision has been built done. Look it.

In [10]:
tree.info

TreeInfo(numPdfs=272, contextWidth=3, centralPosition=1)

Save the tree to file.

In [11]:
treeFile = os.path.join(outDir, "tree")

tree.save(treeFile)

'librispeech_dummy/exp/train_delta/tree'

As mentioned above, we provided a high-level API to build tree directly.

### Train Dicision Tree in high-level API

In [12]:
del tree
os.remove(treeStatsFile)
os.remove(questionsFile)
os.remove(treeFile)

In [13]:
tree = exkaldi.hmm.DecisionTree(lexicons=lexicons,contextWidth=3,centralPosition=1)

tree.train(feat=feat, hmm=hmmFile, ali=ali, topoFile=topoFile, numLeaves=300, tempDir=outDir)

Start to build decision tree.
Start Time: 2020/08/11-13:57:24
Accumulate tree statistics
Cluster phones and compile questions
Build tree
Done to build the decision tree.
Saved Final Tree: librispeech_dummy/exp/train_delta/tree
End Time: 2020/08/11-13:57:27


Tree has been saved in directory automatically.

In [14]:
tree.info

TreeInfo(numPdfs=272, contextWidth=3, centralPosition=1)